### CAP 6640 
### Project 1 - Extractive Summarization
### Feb 8, 2024

### Group 4
### Andres Graterol
###                   UCF ID: 4031393
### Zachary Lyons
###                   UCF ID: 4226832
### Christopher Hinkle
###                   UCF ID: 4038573
### Nicolas Leocadio
###                   UCF ID: 3791733

In [24]:
import string 
import nltk 
import re 
import numpy as np
import networkx as nx

from nltk.tokenize import sent_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from gensim.models import Word2Vec, LsiModel

# Download necessary resources from nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\angel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\angel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Method 1 - TextRank

#### Step 1 - Data Collection

In [25]:
# Gather lengthy articles or a collection of documents that all relate to the same topic (i.e. documents covering an earthquake)
# TextRank: Single-document summarization

'''
    Input: File path to a text file
    Output: String of the text file
'''
def txt_file_to_string(filepath):
    with open(filepath, 'r', encoding='utf8') as file:
        data = file.read()
    return data

# Data is located in text format, character escaped, inside the Documents folder
# TODO: This is a very short sample document to test functionality. When we confirm this works, lets use a larger document.
document_filepath = 'Documents/Japanese_Earthquake-NationalGeographic.txt'
document_text = txt_file_to_string(document_filepath)
print(document_text)

On March 11, 2011, Japan experienced the strongest earthquake in its recorded history. The earthquake struck below the North Pacific, 130 kilometers (81 miles) east of Sendai, the largest city in the Tohoku region, a northern part of the island of Honshu. The Tohoku earthquake caused a tsunami. A tsunami—Japanese for “harbor wave”—is a series of powerful waves caused by the displacement of a large body of water. Most tsunamis, like the one that formed off Tohoku, are triggered by underwater tectonic activity, such as earthquakes and volcanic eruptions. The Tohoku tsunami produced waves up to 40 meters (132 feet) high, More than 450,000 people became homeless as a result of the tsunami. More than 15,500 people died. The tsunami also severely crippled the infrastructure of the country.In addition to the thousands of destroyed homes, businesses, roads, and railways, the tsunami caused the meltdown of three nuclear reactors at the Fukushima Daiichi Nuclear Power Plant. The Fukushima nuclea

#### Step 2 - Data Preprocessing

In [27]:
# TextRank: remove punctuation, tokenize, and remove stopwords

'''
    Purpose: Perform appropriate preprocessing on the text file for the TextRank algorithm
'''
def preprocess_text(text, stop_words):
    tokenized_sentences = sent_tokenize(text, language='english')
    print(tokenized_sentences)

    sentences_to_lower = [sentence.lower() for sentence in tokenized_sentences]
    print(sentences_to_lower)

    # Regular Expression to match any punctuation
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    # Remove the punctuation from the lowercase sentences
    sentences_no_punctuation = [regex.sub('', sentence) for sentence in sentences_to_lower]
    print(sentences_no_punctuation)

    sentence_in_tokens = [[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_no_punctuation]
    return sentence_in_tokens

# Obtain stopwords from nltk
stop_words = set(stopwords.words('english'))
# Preprocess the text to obtain the data we will use going forward
data = preprocess_text(document_text, stop_words)
print(data)

['On March 11, 2011, Japan experienced the strongest earthquake in its recorded history.', 'The earthquake struck below the North Pacific, 130 kilometers (81 miles) east of Sendai, the largest city in the Tohoku region, a northern part of the island of Honshu.', 'The Tohoku earthquake caused a tsunami.', 'A tsunami—Japanese for “harbor wave”—is a series of powerful waves caused by the displacement of a large body of water.', 'Most tsunamis, like the one that formed off Tohoku, are triggered by underwater tectonic activity, such as earthquakes and volcanic eruptions.', 'The Tohoku tsunami produced waves up to 40 meters (132 feet) high, More than 450,000 people became homeless as a result of the tsunami.', 'More than 15,500 people died.', 'The tsunami also severely crippled the infrastructure of the country.In addition to the thousands of destroyed homes, businesses, roads, and railways, the tsunami caused the meltdown of three nuclear reactors at the Fukushima Daiichi Nuclear Power Plan

#### Step 3 - Feature Engineering

In [1]:
# TextRank: Word Embeddings 
# TODO: Take preprocessed data and pass it to Word2Vec to calculate word and sentence embeddings
 
# Train the word 2 vec model
# 
model = Word2Vec(data, min_count=1, size=1, iter=1000)

# TODO: Calculate the similarity matrix


NameError: name 'Word2Vec' is not defined

#### Step 4 - Algorithm and Results


In [ ]:
# TextRank: Call nx's pagerank to get scores. 
# TODO: Make a method that takes the top n scores from the scores variable and grabs the corresponding n sentences

#### Last Step - Evaluation

In [ ]:
# NOTE: Evaluation will depend on the method used to implement extractive summarization
#       - ILP (Integer Linear Programming): We can use ROUGE-2 for evaluation
# Andres NOTE: This is the only section that I am unsure of. It would be cool to use ROUGE-2 to compare our TextRank algorithm to the bigram inspection 


### Method 2 - Latent Semantic Indexing (LSI)

#### Step 1 - Data Collection

In [ ]:
# Gather lengthy articles or a collection of documents that all relate to the same topic (i.e. documents covering an earthquake)
# LSI (Latent Sentiment Indexing): Multi-document summarization

#### Step 2 - Data Preprocessing

In [ ]:
# LSI (Latent Sentiment Indexing): Tokenize, remove stopwords, and stem the words

#### Step 3 - Feature Engineering

In [ ]:
# LSI (Latent Sentiment Indexing): Term Frequency 

#### Step 4 - Algorithm and Results

In [ ]:
# LSI (Latent Sentiment Indexing): Create LSI Model using Gensim

# Sort documents by weight 

# Sort vectors by score 

# Select top documents 

# Sort sentence numbers in order 

# Obtain the summary

#### Last Step - Evaluation

#### References
##### The following tutorials helped us implement the algorithms in the document:
##### 1. https://medium.com/data-science-in-your-pocket/text-summarization-using-textrank-in-nlp-4bce52c5b390
##### 2. https://towardsdatascience.com/document-summarization-using-latent-semantic-indexing-b747ef2d2af6 